# 環境サニティチェック
本ノートでは、Jupyter/pyzmq/GSAS-IIのバージョンと疎通、最小リファインが動くかを検証します。

In [ ]:
# 1) バージョン表示
import sys, zmq, jupyter_client, ipykernel, tornado
print('Python', sys.version)
print('pyzmq', zmq.__version__, 'libzmq', zmq.zmq_version())
print('jupyter_client', jupyter_client.__version__)
print('ipykernel', ipykernel.__version__)
print('tornado', tornado.version_info)
try:
    import jupyter_server, notebook, jupyterlab
    print('jupyter_server', getattr(jupyter_server, '__version__', 'n/a'))
    print('notebook', getattr(notebook, '__version__', 'n/a'))
    print('jupyterlab', getattr(jupyterlab, '__version__', 'n/a'))
except Exception as e:
    print('version check error:', e)

In [ ]:
# 2) 最小GSAS-IIプロジェクトで Scale のみリファイン
from GSASII import GSASIIscriptable as G2sc
import os, math
DATA_DIR = 'data/Y2O3'
cif = os.path.join(DATA_DIR, 'Y2O3.cif')
csv = os.path.join(DATA_DIR, 'Y2O3.csv')
prm = os.path.join(DATA_DIR, 'INST_XRY.PRM')
work = 'work/sanity'; os.makedirs(work, exist_ok=True)
gpx_path = os.path.join(work, 'sanity.gpx')
proj = G2sc.G2Project(newgpx=gpx_path)
hist = proj.add_powder_histogram(csv, prm)
phase = proj.add_phase(cif, phasename='Y2O3', histograms=[hist])
# Scale のみ refine
refdict = {'set': {'Sample Parameters': ['Scale']}}
proj.do_refinements([refdict])
Rwp = hist.get_wR()
print('Rwp =', Rwp)

In [ ]:
# 3) tqdm/ipywidgets の表示確認
from tqdm.auto import tqdm
for _ in tqdm(range(200)):
    pass
print('OK')

In [ ]:
# 4) Rwpが有限かの簡易アサーション
import math
assert math.isfinite(Rwp), 'Rwp is not finite'
print('Sanity OK: Rwp is finite')